# taq_data_load_V4

#### Juan Camilo Henao Londono - 26.03.2019
#### AG Guhr - Universitaet Duisburg-Essen

This version is the same as the V1, just some small adjusments as it was proved I can not find a better implementation.
I removed the `for loop` in the `data_to_array` function.

#### This implementation is NOT used to obtain the TAQ data

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get
import pickle
import matplotlib.pyplot as plt
import os

import sys
sys.path.insert(0, '../Cross_response_individual_stock/taq_algorithms/')

import taq_data_tools

%matplotlib inline

In [2]:
def taq_data_extract(ticker, year, month):
    """
    Extract the quotes and trades data for a month from a CSV file with the
    full information of a year.
        :param ticker: string of the abbreviation of the stock to be analized
                       (i.e. 'AAPL')
        :param year: string of the year to be analized (i.e '2008')
        :param month: string of the month to be analized (i.e '07')
    """
    function_name = taq_data_extract.__name__
    taq_data_tools.taq_function_header_print_data(function_name, ticker,
                                                  ticker, year, month, '-')

    # Load data
    data_quotes = dd.read_csv('../TAQ_{1}/Data/{0}_{1}_NASDAQ_quotes.csv'
                              .format(ticker, year),
                              usecols=range(4),
                              sep=' ',
                              names=['Date', 'Time', 'Bid', 'Ask'],
                              parse_dates=['Date']).set_index('Date')
    data_quotes = data_quotes['{}-{}'.format(year, month)]
    data_quotes = data_quotes.compute()

    data_trades = dd.read_csv('../TAQ_{1}/Data/{0}_{1}_NASDAQ_trades.csv'
                              .format(ticker, year),
                              usecols=range(3),
                              sep=' ',
                              names=['Date', 'Time', 'Ask'],
                              parse_dates=['Date']).set_index('Date')
    data_trades = data_trades['{}-{}'.format(year, month)]
    data_trades = data_trades.compute()

    return (data_quotes, data_trades)

In [3]:
def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [4]:
def taq_data_to_array(ticker, quotes, trades, year, month, day):
    """
    Convert the pandas dataframe information of a month in numpy arrays of a
    day to be used later in the analysis
        :param ticker: string of the abbreviation of the stock to be analized
                       (i.e. 'AAPL')
        :param quotes: pandas dataframe with the quotes information
        :param trades: pandas dataframe with the trades information
        :param year: string of the year to be analized (i.e '2008')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
    """
    function_name = taq_data_to_array.__name__
    taq_data_tools.taq_function_header_print_data(function_name, ticker,
                                                  ticker, year, month, day)

    data_q = quotes.loc[year + '-' + month + '-' + day].copy()
    data_t = trades.loc[year + '-' + month + '-' + day].copy()

    data_q.loc[:, 'Time'] = data_q['Time'].apply(taq_data_tools.get_sec)
    data_t.loc[:, 'Time'] = data_t['Time'].apply(taq_data_tools.get_sec)

    data_q = data_q.loc[(data_q['Time'] >= 34800)
                        & (data_q['Time'] < 57000)]
    data_t = data_t.loc[(data_t['Time'] >= 34800)
                        & (data_t['Time'] < 57000)]

    time_q = np.array(data_q['Time'])
    bid_q = np.array(data_q['Bid'])
    ask_q = np.array(data_q['Ask'])

    time_t = np.array(data_t['Time'])
    ask_t = np.array(data_t['Ask'])
    time_t, ask_t = zip(*sorted(zip(time_t, ask_t)))
    time_t = np.asarray(time_t)
    ask_t = np.asarray(ask_t)

    if (not os.path.isdir('../TAQ_{}/TAQ_py/'.format(year))):

        try:

            os.mkdir('../TAQ_{}/TAQ_py/'.format(year))
            print('Folder to save data created')

        except FileExistsError:

            print('Folder exists. The folder was not created')

    pickle.dump((time_q, bid_q, ask_q),
                open('../TAQ_{1}/TAQ_py/TAQ_{0}_quotes_{1}{2}{3}.pickle'
                     .format(ticker, year, month, day), 'wb'))

    pickle.dump((time_t, ask_t),
                open('../TAQ_{1}/TAQ_py/TAQ_{0}_trades_{1}{2}{3}.pickle'
                     .format(ticker, year, month, day), 'wb'))

    print('Data Saved')
    print()

    return (time_q, bid_q, ask_q, time_t, ask_t)

In [5]:
def months_days_list():
    days_list = []
    months_list = []

    for i in range(1,32):
        if (i < 10):
            days_list.append('0' + str(i))
        else:
            days_list.append(str(i))

    for i in range(1,13):
        if (i < 10):
            months_list.append('0' + str(i))
        else:
            months_list.append(str(i))

    return(months_list, days_list)

In [6]:
tickers = ['AAPL', 'MSFT']
year = '2008'

print(get_sec('09:40:00'))
print(get_sec('15:50:00'))

months_list, days_list = months_days_list()

for ticker in tickers:
    
    for month in months_list:
                
        om_days = days_list[:]
        
        data_quotes, data_trades = taq_data_extract(ticker, year, month)
            
        for d in days_list:

            try:
                (time_q, bid_q, ask_q, time_t, ask_t) = taq_data_to_array(ticker, data_quotes, data_trades, year, month, d)
            except KeyError:
                om_days.remove(d)
                
        pickle.dump(om_days,
                open('days_{}.pickle'
                     .format(month), 'wb'))

34800
57000
TAQ data
taq_data_extract
Processing data for the stock AAPL the 2008.01.-
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.01
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.02
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.03
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.04
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.05
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.06
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.01.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AA

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.01
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.02
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.03
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.04
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.05
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.06
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.04.11
Data Saved

TAQ data
taq_data_to_array
Processing data fo

Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.02
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.03
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.04
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.05
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.06
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.11
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.07.12
TAQ data
taq_data_to_array
Processing data for the stock 

Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.03
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.04
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.05
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.06
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.11
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.12
TAQ data
taq_data_to_array
Processing data for the stock AAPL the 2008.10.13
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock 

Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.04
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.05
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.06
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.11
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.12
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.13
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.01.14
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock 

Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.05
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.06
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.11
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.12
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.13
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.14
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.04.15
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock 

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.06
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.11
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.12
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.13
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.14
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.15
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.07.16
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock 

Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.07
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.08
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.09
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.10
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.11
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.12
TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.13
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.14
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.15
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.16
Data Saved

TAQ data
taq_data_to_array
Processing data for the stock MSFT the 2008.10.17
Data Saved

TAQ data
taq_data_to_array
Proces